# 설문결과 대시보드로 시각화 하기

## 데이터 불러오기

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("data.csv")
df.head()

,시간,참관여부,학년,반,만족도,학부모 의견
0,2022. 4. 13 오후 12:12:54,"네, 참관하였습니다.",2학년,4반,4.0,NaN
1,2022. 4. 13 오후 12:13:04,"네, 참관하였습니다.",5학년,4반,4.0,NaN
2,2022. 4. 13 오후 12:13:05,"네, 참관하였습니다.",5학년,4반,4.0,NaN
3,2022. 4. 13 오후 12:13:05,"네, 참관하였습니다.",1학년,5반,3.0,NaN
4,2022. 4. 13 오후 12:13:09,"네, 참관하였습니다.",4학년,2반,3.0,NaN


## 학급별 응답현황 - groupby

In [5]:
# 학년별 만족도 평균
df.groupby("학년").mean()

,만족도
학년,
1학년,3.480000
2학년,3.676471
3학년,3.419355
4학년,3.511628
5학년,3.428571
6학년,3.619048


In [6]:
# 학급별 만족도
df.groupby(["학년", "반"]).mean()

만족도
학년  반            
1학년 1반   3.125000
    2반   3.571429
    3반   3.500000
    4반   3.777778
    5반   3.428571
2학년 1반   4.000000
    2반   3.800000
    3반   3.400000
    4반   3.625000
3학년 1반   2.333333
    2반   4.000000
    3반   3.818182
    4반   3.222222
    5반   2.666667
4학년 1반   4.000000
    2반   3.600000
    3반   3.000000
    4반   3.625000
    5반   3.750000
    6반   3.300000
5학년 1반   3.375000
    2반   2.000000
    3반   4.000000
    4반   3.500000
    5반   3.714286
    도담반  4.000000
6학년 1반   3.666667
    2반   3.750000
    3반   3.714286
    4반   4.000000
    5반   3.333333

In [21]:
# 학년별 응답현황 카운트
df.groupby(["학년"]).size().reset_index(name="응답수")

,학년,응답수
0,1학년,50
1,2학년,34
2,3학년,31
3,4학년,43
4,5학년,28
5,6학년,21


In [22]:
# 학년별 응답현황 카운트
df.groupby(["학년",'반']).size().reset_index(name="응답수")

,학년,반,응답수
0,1학년,1반,8
1,1학년,2반,7
2,1학년,3반,12
3,1학년,4반,9
4,1학년,5반,14
5,2학년,1반,6
6,2학년,2반,10
7,2학년,3반,10
8,2학년,4반,8
9,3학년,1반,3
